# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [31]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import warnings
warnings.filterwarnings("ignore")

# Import API key
from api_keys import geoapify_key

In [32]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [33]:
%%capture --no-display

# Configure the map plot
city_plot = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = 'City',
    alpha = .5
)

# Display the map
city_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [34]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_df = city_data_df.loc[city_data_df['Max Temp'] < 80, :]
ideal_city_df = city_data_df.loc[city_data_df['Max Temp'] > 70, :]
ideal_city_df = city_data_df.loc[city_data_df['Wind Speed'] < 10, :]
ideal_city_df = city_data_df.loc[city_data_df['Cloudiness'] == 0, :]
ideal_city_df = city_data_df.loc[city_data_df['Humidity'] < 50, :]

# Drop any rows with null values
ideal_clean_df = ideal_city_df.dropna()

# Display sample data
ideal_clean_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230
22,22,guiratinga,-16.3494,-53.7583,38.35,14,22,3.10,BR,1666108243
29,29,saint-louis,16.3333,-15.0000,38.80,24,4,1.78,SN,1666108247
30,30,midwest city,35.4495,-97.3967,9.12,33,75,6.17,US,1666108247


### Step 3: Create a new DataFrame called `hotel_df`.

In [35]:
import numpy as np
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_clean_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = np.nan

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,faya,SA,18.3851,42.4509,35,NaN
4,lanzhou,CN,36.0564,103.7922,48,NaN
22,guiratinga,BR,-16.3494,-53.7583,14,NaN
29,saint-louis,SN,16.3333,-15.0000,24,NaN
30,midwest city,US,35.4495,-97.3967,33,NaN


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [36]:
# Set parameters to search for a hotel
radius = 10000
limit = 1


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    params = {
        "apiKey": geoapify_key,
        "limit": limit,
        "categories": "accommodation.hotel"
    }
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
faya - nearest hotel: No hotel found
lanzhou - nearest hotel: 西园宾馆
guiratinga - nearest hotel: No hotel found
saint-louis - nearest hotel: No hotel found
midwest city - nearest hotel: No hotel found
drochia - nearest hotel: Hotel Confort
collierville - nearest hotel: Days Inn & Suites by Wyndham Collierville Germantown Area
nalut - nearest hotel: No hotel found
rafaela - nearest hotel: Parra Hotel & Suites
saint george - nearest hotel: The Advenire, an Autograph Collection Hotel
arman - nearest hotel: No hotel found
zhangye - nearest hotel: Zhangye Hotel 张掖饭店
heihe - nearest hotel: Санкт-Петербург
rio gallegos - nearest hotel: Amérian Río Gallegos Apart & Suites
siavonga - nearest hotel: Lake Safari Lodge Annex
nova olimpia - nearest hotel: Olímpia Hotel
palencia - nearest hotel: Hotel Área de Servicio Los Chopos
yidu - nearest hotel: No hotel found
cedar city - nearest hotel: Baymont Inn & Suites
baruun-urt - nearest hotel: No hotel found
brenham - nearest hotel:

,City,Country,Lat,Lng,Humidity,Hotel Name
0,faya,SA,18.3851,42.4509,35,No hotel found
4,lanzhou,CN,36.0564,103.7922,48,西园宾馆
22,guiratinga,BR,-16.3494,-53.7583,14,No hotel found
29,saint-louis,SN,16.3333,-15.0000,24,No hotel found
30,midwest city,US,35.4495,-97.3967,33,No hotel found
...,...,...,...,...,...,...
554,freeport,US,40.6576,-73.5832,46,The Freeport Inn and Marina
555,bulgan,MN,48.8125,103.5347,40,Bulgan Hotel
557,saryg-sep,RU,51.5000,95.6000,44,No hotel found
560,changli,CN,39.7042,119.1506,33,客房


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [38]:
%%capture --no-display

# Configure the map plot
city_map = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = 'City',
    alpha = .5,
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)